# Combining Heuristic and AutoEncoder Scores

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
hu_df = pd.read_csv("fraud_ScoreAndRanks_HU.csv")
hu_df.head()

,Record,Manhattan_Hu_Score,Euclidean_Hu_Score,Euclidean_Hu_Rank,Manhattan_Hu_Rank
0,1,0.041778,0.022806,767259.0,448548.0
1,2,0.047428,0.019893,546563.0,671523.0
2,3,0.448251,0.243610,1059975.0,1057206.0
3,4,0.225698,0.097479,1036826.0,1040159.0
4,5,0.108974,0.050330,997471.5,1001392.5


In [4]:
ae_df = pd.read_csv("fraud_ScoreAndRanks_AE.csv")
ae_df.head()

,Record,Manhattan Score,Euclidean Score,Euclidean Rank,Manhattan Rank
0,1,0.056584,0.023870,892429.0,919927.0
1,2,0.039952,0.017979,733476.0,738537.0
2,3,0.400472,0.230180,1060936.0,1058333.0
3,4,0.138758,0.061549,1020751.0,1024992.0
4,5,0.099675,0.047285,1001707.5,1001755.5


In [5]:
allScores_df = ae_df.join(hu_df.set_index('Record'), on='Record')

In [9]:
allScores_df.head()

,Record,Manhattan Score,Euclidean Score,Euclidean Rank,Manhattan Rank,Manhattan_Hu_Score,Euclidean_Hu_Score,Euclidean_Hu_Rank,Manhattan_Hu_Rank
0,1,0.056584,0.023870,892429.0,919927.0,0.041778,0.022806,767259.0,448548.0
1,2,0.039952,0.017979,733476.0,738537.0,0.047428,0.019893,546563.0,671523.0
2,3,0.400472,0.230180,1060936.0,1058333.0,0.448251,0.243610,1059975.0,1057206.0
3,4,0.138758,0.061549,1020751.0,1024992.0,0.225698,0.097479,1036826.0,1040159.0
4,5,0.099675,0.047285,1001707.5,1001755.5,0.108974,0.050330,997471.5,1001392.5


In [11]:
#Check for NaNs
allScores_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1070994 entries, 0 to 1070993
Data columns (total 9 columns):
Record                1070994 non-null int64
Manhattan Score       1070994 non-null float64
Euclidean Score       1070994 non-null float64
Euclidean Rank        1070994 non-null float64
Manhattan Rank        1070994 non-null float64
Manhattan_Hu_Score    1070994 non-null float64
Euclidean_Hu_Score    1070994 non-null float64
Euclidean_Hu_Rank     1070994 non-null float64
Manhattan_Hu_Rank     1070994 non-null float64
dtypes: float64(8), int64(1)
memory usage: 73.5 MB


## Manhattan Scores

In [15]:
manScores_df = allScores_df.loc[:, allScores_df.columns.str.contains('Manhattan')]
manScores_df.head()

,Manhattan Score,Manhattan Rank,Manhattan_Hu_Score,Manhattan_Hu_Rank
0,0.056584,919927.0,0.041778,448548.0
1,0.039952,738537.0,0.047428,671523.0
2,0.400472,1058333.0,0.448251,1057206.0
3,0.138758,1024992.0,0.225698,1040159.0
4,0.099675,1001755.5,0.108974,1001392.5


In [21]:
wt_Hu = 0.5
wt_Ae=0.5
wt_Var=(wt_Hu**2)+(wt_Ae**2)
manScores_df['Wt_AeHu_Zscore']=wt_Hu*manScores_df['Manhattan_Hu_Score']+wt_Ae*manScores_df['Manhattan Score']
manScores_df['Wt_AeHu_ZscoreII']=(wt_Hu*manScores_df['Manhattan_Hu_Score']+wt_Ae*manScores_df['Manhattan Score'])/(wt_Var)
manScores_df.head()

C:\Users\Gregy\Anaconda3\Anaconda3_USC\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\Gregy\Anaconda3\Anaconda3_USC\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,Manhattan Score,Manhattan Rank,Manhattan_Hu_Score,Manhattan_Hu_Rank,Wt_AeHu_Zscore,Wt_AeHu_ZscoreII
0,0.056584,919927.0,0.041778,448548.0,0.049181,0.098361
1,0.039952,738537.0,0.047428,671523.0,0.043690,0.087380
2,0.400472,1058333.0,0.448251,1057206.0,0.424361,0.848723
3,0.138758,1024992.0,0.225698,1040159.0,0.182228,0.364455
4,0.099675,1001755.5,0.108974,1001392.5,0.104325,0.208650


In [23]:
manScores_df['Wt_AeHu_Z_Rank'] = manScores_df['Wt_AeHu_Zscore'].rank(ascending=1)
#manScores_df['Wt_AeHu_Z_RankII'] = manScores_df['Wt_AeHu_ZscoreII'].rank(ascending=1)
df.drop(columns=['Wt_AeHu_Z_RankII'])
manScores_df.head()

C:\Users\Gregy\Anaconda3\Anaconda3_USC\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\Gregy\Anaconda3\Anaconda3_USC\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Manhattan Score,Manhattan Rank,Manhattan_Hu_Score,Manhattan_Hu_Rank,Wt_AeHu_Zscore,Wt_AeHu_ZscoreII,Wt_AeHu_Z_Rank,Wt_AeHu_Z_RankII
0,0.056584,919927.0,0.041778,448548.0,0.049181,0.098361,839162.0,839162.0
1,0.039952,738537.0,0.047428,671523.0,0.043690,0.087380,734159.0,734159.0
2,0.400472,1058333.0,0.448251,1057206.0,0.424361,0.848723,1057812.0,1057812.0
3,0.138758,1024992.0,0.225698,1040159.0,0.182228,0.364455,1034865.0,1034865.0
4,0.099675,1001755.5,0.108974,1001392.5,0.104325,0.208650,1001849.5,1001849.5


In [31]:
manScores_df=manScores_df.drop(columns=['Wt_AeHu_Z_RankII'])
manScores_df.head()

,Manhattan Score,Manhattan Rank,Manhattan_Hu_Score,Manhattan_Hu_Rank,Wt_AeHu_Zscore,Wt_AeHu_ZscoreII,Wt_AeHu_Z_Rank
0,0.056584,919927.0,0.041778,448548.0,0.049181,0.098361,839162.0
1,0.039952,738537.0,0.047428,671523.0,0.043690,0.087380,734159.0
2,0.400472,1058333.0,0.448251,1057206.0,0.424361,0.848723,1057812.0
3,0.138758,1024992.0,0.225698,1040159.0,0.182228,0.364455,1034865.0
4,0.099675,1001755.5,0.108974,1001392.5,0.104325,0.208650,1001849.5


In [32]:
manScores_df.insert(0,'Record',manScores_df.index+1)

In [58]:
manScores_df.sort_values('Wt_AeHu_Z_Rank',ascending=False).to_csv('fraud_ScoreAndRanks_Manhattan.csv',index=False)

In [33]:
manScores_df.sort_values('Wt_AeHu_Z_Rank',ascending=False).head(10)

,Record,Manhattan Score,Manhattan Rank,Manhattan_Hu_Score,Manhattan_Hu_Rank,Wt_AeHu_Zscore,Wt_AeHu_ZscoreII,Wt_AeHu_Z_Rank
930997,930998,4060.945709,1070994.0,4063.945709,1070994.0,4062.445709,8124.891418,1070994.0
371905,371906,3972.844119,1070993.0,3975.844119,1070993.0,3974.344119,7948.688238,1070993.0
850050,850051,2192.208369,1070992.0,2191.841758,1070992.0,2192.025063,4384.050127,1070992.0
514814,514815,2141.385337,1070991.0,2144.385337,1070991.0,2142.885337,4285.770674,1070991.0
110314,110315,869.834246,1070990.0,868.834246,1070990.0,869.334246,1738.668492,1070990.0
189514,189515,633.170562,1070989.0,636.170562,1070989.0,634.670562,1269.341123,1070989.0
706902,706903,629.951136,1070988.0,626.951136,1070988.0,628.451136,1256.902272,1070988.0
917471,917472,573.744000,1070987.0,576.744000,1070987.0,575.244000,1150.488000,1070987.0
864376,864377,569.958455,1070986.0,566.958455,1070986.0,568.458455,1136.916910,1070986.0
892315,892316,515.259650,1070985.0,518.259650,1070985.0,516.759650,1033.519299,1070985.0


In [28]:
#len(manScores_df.loc[(manScores_df['Wt_AeHu_Z_Rank'] != manScores_df['Wt_AeHu_Z_RankII'])])

1070994

## Euclidean

In [34]:
eucScores_df = allScores_df.loc[:, allScores_df.columns.str.contains('Euclidean')]
eucScores_df.insert(0,'Record',eucScores_df.index+1)
eucScores_df.head()

,Record,Euclidean Score,Euclidean Rank,Euclidean_Hu_Score,Euclidean_Hu_Rank
0,1,0.023870,892429.0,0.022806,767259.0
1,2,0.017979,733476.0,0.019893,546563.0
2,3,0.230180,1060936.0,0.243610,1059975.0
3,4,0.061549,1020751.0,0.097479,1036826.0
4,5,0.047285,1001707.5,0.050330,997471.5


In [36]:
wt_Hu = 0.5
wt_Ae=0.5
wt_Var=(wt_Hu**2)+(wt_Ae**2)
eucScores_df['Wt_AeHu_Zscore']=wt_Hu*eucScores_df['Euclidean_Hu_Score']+wt_Ae*eucScores_df['Euclidean Score']
eucScores_df['Wt_AeHu_ZscoreII']=(wt_Hu*eucScores_df['Euclidean_Hu_Score']+wt_Ae*eucScores_df['Euclidean Score'])/(wt_Var)
eucScores_df.head()

C:\Users\Gregy\Anaconda3\Anaconda3_USC\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\Gregy\Anaconda3\Anaconda3_USC\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,Record,Euclidean Score,Euclidean Rank,Euclidean_Hu_Score,Euclidean_Hu_Rank,Wt_AeHu_Zscore,Wt_AeHu_ZscoreII
0,1,0.023870,892429.0,0.022806,767259.0,0.023338,0.046676
1,2,0.017979,733476.0,0.019893,546563.0,0.018936,0.037871
2,3,0.230180,1060936.0,0.243610,1059975.0,0.236895,0.473790
3,4,0.061549,1020751.0,0.097479,1036826.0,0.079514,0.159027
4,5,0.047285,1001707.5,0.050330,997471.5,0.048807,0.097614


In [38]:
eucScores_df['Wt_AeHu_Z_Rank'] = manScores_df['Wt_AeHu_Zscore'].rank(ascending=1)

eucScores_df.head()

C:\Users\Gregy\Anaconda3\Anaconda3_USC\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Record,Euclidean Score,Euclidean Rank,Euclidean_Hu_Score,Euclidean_Hu_Rank,Wt_AeHu_Zscore,Wt_AeHu_ZscoreII,Wt_AeHu_Z_Rank
0,1,0.023870,892429.0,0.022806,767259.0,0.023338,0.046676,839162.0
1,2,0.017979,733476.0,0.019893,546563.0,0.018936,0.037871,734159.0
2,3,0.230180,1060936.0,0.243610,1059975.0,0.236895,0.473790,1057812.0
3,4,0.061549,1020751.0,0.097479,1036826.0,0.079514,0.159027,1034865.0
4,5,0.047285,1001707.5,0.050330,997471.5,0.048807,0.097614,1001849.5


In [57]:
eucScores_df.sort_values('Wt_AeHu_Z_Rank',ascending=False).to_csv('fraud_ScoreAndRanks_Euclidean.csv',index=False)

In [39]:
eucScores_df.sort_values('Wt_AeHu_Z_Rank',ascending=False).head(10)

,Record,Euclidean Score,Euclidean Rank,Euclidean_Hu_Score,Euclidean_Hu_Rank,Wt_AeHu_Zscore,Wt_AeHu_ZscoreII,Wt_AeHu_Z_Rank
930997,930998,1560.318205,1070994.0,1561.434922,1070994.0,1560.876563,3121.753126,1070994.0
371905,371906,1522.010974,1070993.0,1522.954244,1070993.0,1522.482609,3044.965217,1070993.0
850050,850051,941.476849,1070992.0,941.170992,1070992.0,941.323921,1882.647841,1070992.0
514814,514815,820.651454,1070991.0,821.563155,1070991.0,821.107305,1642.214609,1070991.0
110314,110315,419.711010,1070990.0,420.062795,1070990.0,419.886902,839.773805,1070990.0
189514,189515,241.782061,1070987.0,242.963549,1070987.0,242.372805,484.745610,1070989.0
706902,706903,296.421642,1070989.0,295.319498,1070989.0,295.870570,591.741140,1070988.0
917471,917472,224.778857,1070985.0,225.575000,1070985.0,225.176929,450.353857,1070987.0
864376,864377,273.110417,1070988.0,273.157981,1070988.0,273.134199,546.268398,1070986.0
892315,892316,198.019193,1070983.0,199.123183,1070983.0,198.571188,397.142376,1070985.0


## Check the original data

In [40]:
NY_df = pd.read_csv("NY property data.csv")

In [42]:
NY_df["Fr_Rank"] = eucScores_df['Wt_AeHu_Z_Rank']
#NY_df["Fr_Rank"] = eucScores_df['Wt_AeHu_Z_Rank']
#NY_df["Fr_Rank"] = eucScores_df['Wt_AeHu_Z_Rank']
NY_df.head()

,RECORD,BBLE,B,BLOCK,LOT,EASEMENT,OWNER,BLDGCL,TAXCLASS,LTFRONT,...,BLDDEPTH,AVLAND2,AVTOT2,EXLAND2,EXTOT2,EXCD2,PERIOD,YEAR,VALTYPE,Fr_Rank
0,1,1000010101,1,1,101,NaN,U S GOVT LAND & BLDGS,P7,4,500,...,0,3775500.0,8613000.0,3775500.0,8613000.0,NaN,FINAL,2010/11,AC-TR,839162.0
1,2,1000010201,1,1,201,NaN,U S GOVT LAND & BLDGS,Z9,4,27,...,0,11111400.0,80690400.0,11111400.0,80690400.0,NaN,FINAL,2010/11,AC-TR,734159.0
2,3,1000020001,1,2,1,NaN,DEPT OF GENERAL SERVI,Y7,4,709,...,564,32321790.0,40179510.0,32321790.0,40179510.0,NaN,FINAL,2010/11,AC-TR,1057812.0
3,4,1000020023,1,2,23,NaN,DEPARTMENT OF BUSINES,T2,4,793,...,551,13644000.0,15750000.0,13644000.0,15750000.0,NaN,FINAL,2010/11,AC-TR,1034865.0
4,5,1000030001,1,3,1,NaN,PARKS AND RECREATION,Q1,4,323,...,57,106348680.0,107758350.0,106348680.0,107758350.0,NaN,FINAL,2010/11,AC-TR,1001849.5


In [45]:
NY_FraudSorted = NY_df.sort_values('Fr_Rank',ascending=False)

In [46]:
NY_FraudSorted.head()

,RECORD,BBLE,B,BLOCK,LOT,EASEMENT,OWNER,BLDGCL,TAXCLASS,LTFRONT,...,BLDDEPTH,AVLAND2,AVTOT2,EXLAND2,EXTOT2,EXCD2,PERIOD,YEAR,VALTYPE,Fr_Rank
930997,930998,4161780060,4,16178,60,NaN,NYS DOT,U7,3,9,...,0,NaN,NaN,NaN,NaN,NaN,FINAL,2010/11,AC-TR,1070994.0
371905,371906,3034280142,3,3428,142,NaN,YDELFONSO R REYES,B2,1,19,...,50,NaN,NaN,NaN,NaN,NaN,FINAL,2010/11,AC-TR,1070993.0
850050,850051,4109900039,4,10990,39,NaN,"BRYAN-NELSON, VERONIC",B3,1,30,...,26,NaN,NaN,NaN,NaN,NaN,FINAL,2010/11,AC-TR,1070992.0
514814,514815,3073100016,3,7310,16,NaN,ANN GILL,A1,1,26,...,42,NaN,NaN,NaN,NaN,NaN,FINAL,2010/11,AC-TR,1070991.0
110314,110315,1014960021,1,1496,21,NaN,ALBERT JOSEPH,D6,2,102,...,125,1948500.0,10017900.0,NaN,NaN,NaN,FINAL,2010/11,AC-TR,1070990.0


In [49]:
NY_FraudSorted.to_csv('NY_FraudSorted.csv',index=False)

In [47]:
fraud10 = ['930998',
            '371906',
            '850051',
            '514815',
            '110315',
            '189515',
            '706903',
            '917472',
            '864377',
            '892316']

In [48]:
fraud10

['930998',
 '371906',
 '850051',
 '514815',
 '110315',
 '189515',
 '706903',
 '917472',
 '864377',
 '892316']

In [52]:
#NY_df.loc[NY_df['RECORD'].isin(fraud10)]

In [54]:
cols = ["RECORD",
        "Fr_Rank",
        "LTFRONT",
        "LTDEPTH",
        "BLDFRONT",
        "BLDDEPTH",
        "STORIES",
        "OWNER",
        "BLDGCL",
        "TAXCLASS",
        "ZIP"
       ]

In [56]:
NY_FraudSorted[cols].head(10)

,RECORD,Fr_Rank,LTFRONT,LTDEPTH,BLDFRONT,BLDDEPTH,STORIES,OWNER,BLDGCL,TAXCLASS,ZIP
930997,930998,1070994.0,9,192,0,0,NaN,NYS DOT,U7,3,NaN
371905,371906,1070993.0,19,119,19,50,2.0,YDELFONSO R REYES,B2,1,11207.0
850050,850051,1070992.0,30,100,20,26,2.5,"BRYAN-NELSON, VERONIC",B3,1,11412.0
514814,514815,1070991.0,26,100,16,42,2.0,ANN GILL,A1,1,11229.0
110314,110315,1070990.0,102,125,91,125,11.0,ALBERT JOSEPH,D6,2,10028.0
189514,189515,1070989.0,0,0,0,0,8.0,"LEBRON, MARIA",R4,2,10462.0
706902,706903,1070988.0,40,100,23,54,1.0,"SEO, HYUN",A2,1,11354.0
917471,917472,1070987.0,40,80,28,35,2.0,"VASQUEZ, DANIEL",B1,1,11414.0
864376,864377,1070986.0,0,0,0,0,1.0,AGUAYO EDNA,R3,1A,11417.0
892315,892316,1070985.0,30,105,18,32,2.5,"ALI, SHAHEED W",A1,1,11413.0
